## Programming Assignment: Regularized Logistic Regression

Chào mừng các bạn đến với bài tập lập trình Regularized Logistic Regression (Bài toán phân loại nhị phân - 2 nhóm). Trước khi thực hiện bài tập này, các bạn nên học kỹ các kiến thức lý thuyết. Nếu có bất kỳ câu hỏi hay vấn đề nào xảy ra, các bạn hãy để lại comment trực tiếp bên dưới bài đăng hoặc liên hệ qua Fanpage AIVIETNAM.

### Hướng dẫn làm bài 
- Trong bài tập này bạn sẽ sử dụng Python 3.
- Cố gắng không sử dụng các vòng lặp (for, while). 
- Hãy sử dụng các hàm của thư viện numpy.
- Sau khi bạn viết Code của mình xong, hãy chạy dòng Code đó để xem kết quả bên dưới. 

Các bạn sẽ bắt đầu Code trong phần `### START CODE HERE ###` và `### END CODE HERE ###`. Các bạn nhớ đừng sửa bất kỳ dòng Code nào bên ngoài những câu lệnh này. 

Sau khi viết xong Code của bạn, bạn hãy ấn "SHIFT"+"ENTER" để thực hiện chạy lệnh của Cell đó. 

Trong phần Code: các bạn hãy cố gắng thực hiện ít dòng Code nhất theo chỉ định "(≈ X lines of code)". Mặc dù đây không phải là hạn chế về số dòng Code của bạn, nhưng hãy tối ưu sao cho ít nhất có thể.

### Chú ý

Trong phần bài tập này, chúng ta sẽ sử dụng **Advanced Optimization**

Ưu điểm của phương pháp này:
- Không cần phải chọn **learning_rate** (có một vòng lặp bên trong thuật toán để tìm ra giá trị learning_rate tốt nhất).
- Thường hội tụ nhanh hơn Gradient Descent.
- Dễ dàng sử dụng mà không cần tìm hiểu quá sâu bên trong.

Nhược điểm:
- Không nên tự thực hiện mà hãy sử dụng thư viện có sẵn.
- Nếu có lỗi xảy ra việc sửa lỗi thường phức tạp.

In [ ]:
# Import thư viện 
# Standard imports. Importing seaborn for styling.
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn; seaborn.set_style('whitegrid')

### Regularized logistic regression

Trong phần bài tập này, Regularized Logistic Regression sẽ được sử dụng để dự đoán các vi mạch chế tạo từ nhà máy có vượt qua kiểm duyệt hay không?

### 1. Trực quan hoá dữ liệu 

Như trong phần bài tập Logistic Regression, chúng ta sẽ nhập dữ liệu và trực quan hoá.

In [ ]:
data = np.loadtxt('data/data2.txt', delimiter=',')
X, y = data[:,:2], data[:,2]

# Viewing the imported values (first 5 rows)
X[:5], y[:5]

In [ ]:
# Hàm trực quan dữ liệu 
# Creating plotData method to display the figure where the axes are the two exam scores.
def plotData(x, y, xlabel, ylabel, labelPos, labelNeg):
    
    # Separating positive and negative scores (in this case 1 and 0 values):
    pos = y==1
    neg = y==0

    # Scatter plotting the data, filtering them according the pos/neg values:
    plt.scatter(x[pos, 0], x[pos, 1], s=30, c='darkblue', marker='+', label=labelPos)
    plt.scatter(x[neg, 0], x[neg, 1], s=30, c='yellow', marker='o', edgecolors='y', label=labelNeg)

    # Labels and limits:
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.xlim(x[:, 0].min(), x[:, 0].max())
    plt.ylim(x[:, 1].min(), x[:, 1].max())

    # Legend:
    pst = plt.legend(loc='upper right', frameon=True)
    pst.get_frame().set_edgecolor('k');

In [ ]:
"""
Accepted: Được chấp nhận.
Rejected: Không được chấp nhận.
"""

plotData(X, y, 'Microchip Test 1', 'Microchip Test 2', 'Accepted', 'Rejected')

### 2. Feature mapping

Trong phần này, chúng ta sẽ sử dụng Polynomial Logistic Regression để có thể tìm được decision boundary phân cách 2 nhóm dữ liệu. Hàm __PolynomialFeatures__ từ thư viện __scikit-learn__ sẽ giúp chúng ta thực hiện.

Ví dụ về cách sử dụng __PolynomialFeatures__ đã được đề cập trong phần bài toán Polynomial Linear Regression, xem lại [tại đây](https://nbviewer.jupyter.org/github/thanhhff/AIVN-Machine-Learning/blob/master/Week%203/Polynomial-regression.ipynb).

In [ ]:
# Importing PolynomialFeatures
from sklearn.preprocessing import PolynomialFeatures

# Creating the model
# Đa thức bậc 6
poly = PolynomialFeatures(6)

# Chuyển đổi dữ liệu thành đa thức bậc 6
X2 = poly.fit_transform(X)
X2.shape

### 3. Cost function và Gradient

Trong phần này bạn sẽ viết hàm __cost function__ và __gradient methods__ cho Regularized logistic regression.

#### 1. Cost Function 

**Bài tập:** Viết hàm Cost.

> $h = g(X\theta)$ 

**Chú ý:** do lỗi làm tròn số nên giá trị khi tính `Sigmoid` khiến `log(0)` không xác định. Nên để loại bỏ lỗi này ta thêm giá trị `eps = 1e-15` (một số nhỏ vừa đủ) vào công thức `J`:

> $J(\theta) = - \frac{1}{m} [ y^T \log{h} + (1-y)^T\log{(1 - h + eps)} ] + \frac{\lambda}{2m} \sum_{j = 1}^{n} \theta_j^2$

**Chú ý:** không đánh phạt hệ số $\theta_0$

In [ ]:
from scipy.special import expit
# Sử dụng hàm sigmoid trong thư viện scipy 
def sigmoid(z):
    return expit(z)

In [ ]:
# Viết hàm regularized costFunction:
def costFunctionR(theta, X, y, lam):
    # Số lượng training
    ### START CODE HERE ### (≈ 1 line of code)
    m = None
    ### END CODE HERE ###
    
    eps = 1e-15
    
    ### START CODE HERE ### (≈ 2 line of code)
    hThetaX = None
    J = None
    ### END CODE HERE ###
    
    return J

#### 2. Gradient methods 

**Bài tập:** Viết Gradient Methods 

> $gradient = \frac{1}{m} X^T (g(X\theta) - \vec{y}) + \lambda\theta' = \frac{1}{m} X^T (h - \vec{y}) + \lambda\theta'$

Trong đó:
- $\theta'$ có giá trị $\theta_0 = 0$ (vì không đánh phạt $\theta_0$), các giá trị còn lại tương tự với $\theta$.

**Gợi ý:** hàm [numpy.insert](https://docs.scipy.org/doc/numpy/reference/generated/numpy.insert.html)

``` 
numpy.insert(arr, obj, values, axis=None)

arr: input_array 
obj: vị trí cần chèn 
values: giá trị 
```

In [ ]:
# Viết hàm regularized gradientFunc:
def gradientFuncR(theta, X, y, lam):
    ### START CODE HERE ### (≈ 4 line of code)
    
    m = None
    hThetaX = None
    
    # Không regularizing tham số θ0 nên sẽ thay θ0 = 0  
    thetaNoZeroReg = None
    
    gradient = None 
    ### END CODE HERE ###
    
    return gradient

In [ ]:
# Khởi tạo theta
initial_theta = np.zeros(X2.shape[1])
initial_theta

Gọi hàm __*costFunctionR*__ và __*gradientFuncR*__ sử dụng tham số θ được khởi tạo ở trên, với giá trị lambda = 1.

In [ ]:
### START CODE HERE ### (≈ 2 line of code)
J = None
gradient = None
### END CODE HERE ###

# Giá trị Cost khoảng 0.693 cho phần này
print("Cost: %0.3f"%(J))
print("Gradient: {0}".format(gradient))

**Đầu ra kỳ vọng:**

```
Cost: 0.693
Gradient: [8.47457627e-03 1.87880932e-02 7.77711864e-05 5.03446395e-02
 1.15013308e-02 3.76648474e-02 1.83559872e-02 7.32393391e-03
 8.19244468e-03 2.34764889e-02 3.93486234e-02 2.23923907e-03
 1.28600503e-02 3.09593720e-03 3.93028171e-02 1.99707467e-02
 4.32983232e-03 3.38643902e-03 5.83822078e-03 4.47629067e-03
 3.10079849e-02 3.10312442e-02 1.09740238e-03 6.31570797e-03
 4.08503006e-04 7.26504316e-03 1.37646175e-03 3.87936363e-02]

```

### 3. Tìm tham số theta bằng scipy.optimize sử dụng .minimize

Thực hiện tìm tham số theta bằng __scipy.optimize__ sử dụng __*.minimize__, như trong phần bài tập __2.3__ (Logistic Regression).

```
scipy.optimize.minimize(fun, x0, args=(), method=None, jac=None, hess=None, hessp=None, bounds=None, constraints=(), tol=None, callback=None, options=None)[source]
```

Các bạn sẽ cài đặt một số tham số dưới đây (ngoài ra không cần thiết).

```
- fun: costFunctionR
- x0: initial_theta
- args: (X2, y, 1) - vì trong bài này sử dụng Regularized nên đối số 1 dành cho lambda
- method: trong bài này chúng ta sử dụng BFGS
- jac: gradientFuncR 
- options={'maxiter' : 400, 'disp': True} với maxiter: số lần lặp tối đa; disp: hiển thị thông tin
```

In [ ]:
# Importing minimize from scipy:
from scipy.optimize import minimize

### START CODE HERE ### (≈ 2 line of code)
result2 = None
### END CODE HERE ###

result2['x']

**Đầu ra kỳ vọng:**
```
Optimization terminated successfully.
         Current function value: 0.529003
         Iterations: 47
         Function evaluations: 48
         Gradient evaluations: 48
array([ 1.27268739,  0.62557016,  1.1809665 , -2.01919822, -0.91761468,
       -1.43194199,  0.12375921, -0.36513086, -0.35703388, -0.17485805,
       -1.45843772, -0.05129676, -0.61603963, -0.2746414 , -1.19282569,
       -0.24270336, -0.20570022, -0.04499768, -0.27782709, -0.29525851,
       -0.45613294, -1.04377851,  0.02762813, -0.29265642,  0.01543393,
       -0.32759318, -0.14389199, -0.92460119])
```

### 4. Vẽ decision boundary

In [ ]:
def plotDecisionBoundary(X, y, title):
    # Plot the data
    plotData(X[:, 1:3], y, 'Microchip Test 1', 'Microchip Test 2', 'Accepted', 'Rejected')
    
    # Defining the data to use in the meshgrid calculation. Outputting xx and yy ndarrays
    x_min, x_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    y_min, y_max = X[:, 2].min() - 1, X[:, 2].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02), np.arange(y_min, y_max, 0.02))
    
    Z = sigmoid(poly.fit_transform(np.c_[xx.ravel(), yy.ravel()]).dot(result2['x']))
    Z = Z.reshape(xx.shape)
    
    # Plotting the contour plot
    plt.contour(xx, yy, Z, [0.5], linewidths=1, colors='g')
    plt.title(title)

### 5. Thay đổi giá trị của λ

Trong phần bài tập này, các bạn sẽ thấy nếu với λ khác nhau trong thì decision boundary sẽ thay đổi như thế nào. Chúng ta sẽ bị overfitting nếu không sử dụng regularization tức λ = 0. Mặt khác, nếu λ quá lớn sẽ bị underfitting.

In [ ]:
plt.figure(figsize=(6, 15))
plt.subplots_adjust(hspace=0.3)

# Creating 3 subplots using 3 different λ values
for i, lam in enumerate([0, 1, 100]):
    result2 = minimize(costFunctionR, initial_theta, args=(X2, y, lam), method='BFGS', jac=gradientFuncR, 
                       options={'maxiter' : 400, 'disp': False})
    
    if (lam == 0):
        title = 'No regularization (Overfitting) (λ = 0)'
    elif (lam == 100):
        title = 'Too much regularization (Underfitting) (λ = 100)'
    else:
        title = 'Training data with decision boundary (λ = 1)'
    
    plt.subplot(3, 1, i+1)
    
    # Plotting the decision boundary plot
    plotDecisionBoundary(X2, y, title);

### Tổng kết

Thông qua bài tập này, các bạn đã nắm vững các kiến thức về:

- Regularized Logistic Regression
- Triển khai hàm Cost Function và Gradient Method sử dụng BFGS
- Đánh giá sự khác biệt khi sử dụng Regularized.
- Thay đổi giá trị của lambda khác nhau.

### Tài liệu tham khảo

[1] [CS229 - Machine Learning]()